In [22]:
import os
import torch

dataset = "MNIST"
project_name = "MTVAEs_05.01-cross-val"
directory = f"assets/results/raw/{project_name}/{dataset}/"

class Loss:
    def __init__(self, loss_name: str, average: float, std: float):
        self.loss_name = loss_name
        self.average = average
        self.std = std    

class Result:
    def __init__(self, filename: str, data: dict[str, Loss]):
        self.filename = filename
        self.data = data
        
def get_crossval_results(data: list)-> dict[str, Loss]:
    losses: dict[str, Loss]= {}
    
    first : dict = data[0][0]
    
    keys = first.keys()
    
    for key in keys:
        values = []
        for item in data:
            values.append(item[0][key])
            
        average = sum(values) / len(values)
        std = sum([(value - average)**2 for value in values]) / len(values)
        print (f"{key}: {average} ± {std}")
        
        losses[key] = Loss(key, average, std)
        
    return losses

def get_results(directory: str) -> list[Result]:
    results = []
    print("Loading results...")
    for filename in os.listdir(directory):
        if filename.endswith(".pt"):
            # load the file
            data = torch.load(directory + filename)
            results.append(Result(filename, data))
            #print(f"Loaded {filename}")
            
    print("Loaded: ", len(results))
    print("Done.")
    return results
      
results = get_results(directory)

Loading results...
Loaded:  45
Done.
